# Classificação com BERT

Neste *notebook* você verá um exemplo de como usar o [BERT](https://arxiv.org/abs/1810.04805) para tarefa de classificação, usando a bilbioteca **Transformers** do **Hugging Faces**. 

Fontes:  

- [BERT Fine-Tuning Tutorial with PyTorch](https://mccormickml.com/2019/07/22/BERT-fine-tuning/).
- [Hugging Faces - Github](https://github.com/huggingface/transformers) e [Hugging Faces - site](https://huggingface.co/)


Primeiro, vamos verificar se temos GPU disponível para nossa execução. Não se preocupe caso não possua GPU, apenas o treinamento será mais demorado.

Caso você esteja executando no Colab, acesse: Edit 🡒 Notebook Settings 🡒 Hardware accelerator 🡒 (GPU)

In [ ]:
!pip install accelerate
!pip install bitsandbytes
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
import torch

# Verificar se tem GPU disponível
if torch.cuda.is_available():    

    # Informa PyTorch para usar GPU
    device = torch.device("cuda")

    print('Existe(m)  %d GPU(s) disponível(eis).' % torch.cuda.device_count())

    print('Vamos usar a GPU:', torch.cuda.get_device_name(0))

# Se não tem...
else:
    print('Sem GPU disponível, usando CPU.')
    device = torch.device("cpu")

### Biblioteca Transformers com *Pytorch*

Aqui vamos usar a interface *Pytorch* porque possui um bom equilíbrio entre as APIs de alto nível (fáceis de usar, mas sem fornecer informações sobre como as coisas funcionam) e código de tensorflow (que contém muitos detalhes, mas não tão fáceis de usar).

No momento, a biblioteca **Hugging Face** é a interface *Pytorch* mais utilizada para trabalhar com BERT. Além de oferecer suporte a uma variedade de modelos pré-treinados, também inclui modelos especializados para tarefas específicas. Neste tutorial vamos usar *BertForSequenceClassification*.

A biblioteca também inclui classes específicas de tarefas para classificação de *tokens*, resposta a perguntas (Q&A), previsão da próxima frase, etc. O uso dessas classes pré-construídas simplifica o processo de modificação de BERT para nossos propósitos.

### Análise

Podemos ver pelos nomes dos arquivos que temos as versões originais e as tokenizadas dos dados.

Não vamos usar a versão pré-tokenizada porque, para aplicar o nosso modelo BERT pré-treinado, devemos usar o tokenizador fornecido pelo modelo. Isso porque: (1) o modelo tem um vocabulário específico e fixo e (2) o tokenizador BERT tem uma maneira particular de lidar com palavras fora do vocabulário.

Usaremos a biblioteca *pandas* para analisar o conjunto de treinamento e examinar algumas de suas propriedades.

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

# Carrega o dataset em um dataframe pandas
df = pd.read_csv("../dataset/cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

# Imprime o número de frases.
print('Número de sentenças de treinamento: {:,}\n'.format(df.shape[0]))

# Exibe 10 frases randômicas do dataset
df.sample(10)

As duas propriedades importantes são as frases (*sentence*) e seus rótulos (*label*), que podem ser 0 = incorreta e 1 = correta.

Aqui estão cinco frases que são rotuladas como incorretas gramaticalmente (em inglês). Observe como essa tarefa é muito mais difícil do que algo como análise de sentimento!

In [ ]:
df.loc[df.label == 0].sample(5)[['sentence', 'label']]

Vamos extrair as frases e seus rótulos do nosso corpus de treinamento como *numpy ndarrays*.

In [ ]:
# Lista de sentenças e seus rótulos
sentences = df.sentence.values
labels = df.label.values

### Tokenização

Agora vamos transformar nosso conjunto de dados no formato em que o BERT pode ser treinado.

Para alimentar o BERT com nosso texto, ele deve ser dividido em *tokens* e, em seguida, esses *tokens* devem ser mapeados para seu índice no vocabulário do tokenizador.

A tokenização deve ser realizada pelo tokenizador incluído no modelo BERT que estamos trabalhando, que vamos baixar no código abaixo.

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer

# Carregar o tokenizador BERT.
print('Carregando tokenizador BERT...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Vamos aplicar o tokenizador a uma frase para ver a saída.

In [ ]:
# Frase original
print(' Original: ', sentences[0])

# Frase dividida em tokens
print('Tokenizada: ', tokenizer.tokenize(sentences[0]))

# Sentença mapeada em token ids
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

Quando realmente convertermos todas as nossas sentenças, usaremos a função ```tokenize.encode``` para lidar com ambas as etapas, em vez de chamar as funções que usamos acima separadamente.

Antes de fazermos isso, precisamos falar sobre alguns dos requisitos de formatação do BERT.

### Formatação

O código acima omitiu algumas etapas de formatação necessárias que veremos aqui.

Precisamos:
1. Adicionar *tokens* especiais no início e fim de cada frase.
2. Prencher e truncar todas as frases para terem o mesmo comprimento.
3. Diferenciar explicitamente *tokens* reais de *tokens* de preenchimento (PAD) com a “máscara de atenção”.

**Tokens especiais**
[SEP] - No final de cada frase, precisamos acrescentar o token especial [SEP], que é usado em tarefas de duas sentenças, onde BERT recebe duas sentenças separadas e é solicitado a determinar algo (por exemplo, a resposta à pergunta na sentença A pode ser encontrada na sentença B).

[CLS] - Para tarefas de classificação, devemos acrescentar o [CLS] ao início de cada frase. Como o BERT consiste em 12 camadas de Tranformer, cada uma recebe uma lista de *embeddings* e produz o mesmo número de *embeddings* na saída (com outros valores), na saída do transformador final (12º), apenas o primeiro *embedding* (correspondente ao *token* [CLS]) é usado pelo classificador .

“O primeiro *token* de cada sequência é sempre um *token* de classificação especial ([CLS]), onde o estado oculto final deste *token* é usado como a representação da sequência agregada para tarefas de classificação.” (do artigo BERT)

Não precisamos pensar em estratégias de *pool* sobre os *embeddings* finais, pois nesse token [CLS] de classificação, o modelo codificou tudo o que precisamos para a classificação naquele único vetor de incorporação de 768 valores. Já está feito o *pool* para nós!

*Comprimento da frase e máscara de atenção*

As sentenças em nosso conjunto de dados obviamente têm comprimentos variados, então como o BERT lida com isso?

BERT tem duas restrições:

- Todas as frases devem ser preenchidas ou truncadas em um único comprimento fixo.
- O comprimento máximo da frase é 512 *tokens*.

O preenchimento é feito com um *token* especial [PAD], que está no índice 0 no vocabulário BERT.

A “Máscara de Atenção” é simplesmente uma matriz de 1s e 0s indicando quais *tokens* são de preenchimento e quais não são (parece meio redundante, não é ?). Essa máscara diz ao mecanismo de “Autoatenção” do BERT para não incorporar esses *tokens* (PAD) em sua interpretação da frase.

O comprimento máximo afeta a velocidade de treinamento e avaliação. Por exemplo, com um Tesla K80:

MAX_LEN = 128 --> Cada época leva ~5:28 para treinar

MAX_LEN = 64 --> Cada época leva ~2:57 para treinar

**Tokenizar o conjunto de dados**

A biblioteca Transformers fornece uma função ```encode``` que tratará da maioria das etapas de análise e preparação de dados para nós.

Antes de estarmos prontos para codificar nosso texto, precisamos decidir sobre o comprimento máximo de frase para preenchimento ou truncamento.

A célula abaixo realizará uma passagem de tokenização em nosso conjunto de dados para medir o comprimento máximo da frase.

In [ ]:
max_len = 0

# Para cada frase
for sent in sentences:

    # Tokeniza o texto e adiciona os tokens `[CLS]` e `[SEP]`
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Atualiza o comprimento máximo da frase
    max_len = max(max_len, len(input_ids))

print('Frase com tamanho máximo no nosso dataset: ', max_len)

Para o caso de haver algumas sentenças mais longas no conjunto de teste, vams setar nosso comprimento máximo em 64.

Agora estamos prontos para realizar a tokenização.

A função ```tokenizer.encode_plus``` realiza várias etapas para nós:

1. Divide a frase em *tokens*.
2. Adiciona os *tokens* especiais [CLS] e [SEP].
3. Mapeia os *tokens* para seus IDs.
4. Preenche ou trunca todas as frases com o mesmo comprimento.
5. Cria as máscaras de atenção que diferenciam explicitamente *tokens* reais de *tokens* [PAD].

As primeiras quatro etapas são realizadas pela função ```tokenizer.encode```, mas vamos usar a ```tokenizer.encode_plus``` para a quinta etapa (máscaras de atenção). [Documentação aqui](https://huggingface.co/transformers/main_classes/tokenizer.html?highlight=encode_plus#transformers.PreTrainedTokenizer.encode_plus)

In [ ]:
# Tokeniza todas as frases e mapeia os tokens em seus IDs
input_ids = []
attention_masks = []

# Para cada frase
for sent in sentences:
    # `encode_plus` fará o seguinte:
    #   (1) Tokeniza a frase
    #   (2) Adiciona o token `[CLS]` no inicio
    #   (3) Adiciona o token `[SEP]` no final
    #   (4) Mapeia os tokens em seus IDs.
    #   (5) Adiciona preenchimento (pad) ou trunca a frase atá o comprimento máximo (`max_length`)
    #   (6) Cria máscara de atenção para os tokens [PAD].
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Frase a ser codificada
                        add_special_tokens = True, # Adiciona '[CLS]' e '[SEP]'
                        max_length = 64,           # Preenche & trunca todas as frases
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Constrói máscaras de atenção
                        return_tensors = 'pt',     # Returna tensores pytorch.
                   )
    
    # Adiciona a frase codificada na lista
    input_ids.append(encoded_dict['input_ids'])
    
    # E sua máscara de atenção (simplesmente diferencia tokens reais de tokens de preenchimento - PAD).
    attention_masks.append(encoded_dict['attention_mask'])

# Converte a lista em tensores
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Imprime frase 0, agora como uma lista de IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

### Divisão dos dados - treinamento e validação

Precisamos agora dividir nosso conjunto de treinamento, em 90% para treinamento e 10% para validação.


In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combina as entradas de treinamento em um TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Criar uma divisão 90-10 para treinamento-validação.

# Calcula o número de instâncias para incluir em cada divisão
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide o dataset pegando randomicamente as instância 
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} instâncias de treinamento'.format(train_size))
print('{:>5,} instâncias de validação'.format(val_size))

Vamos criar um iterador para nosso conjunto de dados usando a classe *DataLoader*, para economizar memória durante o treinamento, visto que, ao contrário de um *loop* com *for*, com o iterador o conjunto de dados inteiro não precisa ser carregado na memória.

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# O DataLoader precisa saber no tamanho do batch size para treinamento, então vamos espeficiar
# Para o fine-tuning em uma tarefa, os autores recomendam um batch size de 16 ou 32.
batch_size = 32

# Criar os DataLoaders para nossos conjuntos de treinamento e teste
# Vamos pegar instâncias de treinamento em ordem aleatória
train_dataloader = DataLoader(
            train_dataset,  # Exemplos de treinamento
            sampler = RandomSampler(train_dataset), # Seleciona batches aleatoriamente
            batch_size = batch_size # Treina com este batch size.
        )

# Para validação, a ordem não importa, então vamos mantê-la sequencial
validation_dataloader = DataLoader(
            val_dataset, # Exemplos de validação.
            sampler = SequentialSampler(val_dataset), # Executa os batches sequencialmente.
            batch_size = batch_size # Avalia com este batch size.
        )

## Modelo de classificação

Agora que nossos dados de entrada estão formatados corretamente, é hora de especializar o modelo BERT.

### BertForSequenceClassification

Para a tarefa de classificação, vamos modificar o modelo BERT pré-treinado para fornecer saídas para classificação e, em seguida, treinar todo modelo em nosso conjunto de dados até que esteja especializado para nossa tarefa.

Felizmente, a implementação *Pytorch* do Hugging Face inclui um conjunto de interfaces projetadas para uma variedade de tarefas de PLN. Embora essas interfaces sejam todas construídas em cima de um modelo BERT treinado, cada uma tem diferentes camadas superiores e tipos de saída projetados para nossa tarefa PLN específica.

Aqui está a lista atual de classes fornecidas para a especialização da tarefa (ajuste fino ou *fine tuning*):

1. BertModel
1. BertForPreTraining
1. BertForMaskedLM
1. BertForNextSentencePrediction
1. BertForSequenceClassification (**usaremos aqui**)
1. BertForTokenClassification
1. BertForQuestionAnswering

A documentação para eles pode ser encontrada [aqui](https://huggingface.co/transformers/v2.2.0/model_doc/bert.html).

Estaremos usando *BertForSequenceClassification*, modelo BERT com uma camada linear única adicionada no topo para classificação, que será usado como um classificador de frases. A medida que alimentamos os dados de entrada, todo o modelo BERT pré-treinado e a camada adicional de classificação não treinada serão treinados em nossa tarefa específica.

OK, vamos carregar o BERT! Existem alguns modelos diferentes de BERT pré-treinados disponíveis. “bert-base-uncased”, modell em inglês, significa a versão que tem apenas letras minúsculas (“sem caixa”) e é a versão menor das duas (“base” vs “large”).

A documentação do método ```from_pretrained``` pode ser encontrada [aqui](https://huggingface.co/transformers/v2.2.0/main_classes/model.html#transformers.PreTrainedModel.from_pretrained), com os parâmetros adicionais definidos [aqui](https://huggingface.co/transformers/v2.2.0/main_classes/configuration.html#transformers.PretrainedConfig).

In [ ]:
from transformers import BertForSequenceClassification, BertConfig
from torch.optim import AdamW

# Carrega a classe BertForSequenceClassification, o modelo pre-treinado com uma camada linear simples no topo
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Usa o BERT com 12 camadas, com vocabulario com caixa baixa
    num_labels = 2, # O número de saídas, ou rótulos, do nosso modelo (classificação binária em nosso caso)
                    # Para tarefas com mais classes (multi-classe), podemos aumentar esse número
    output_attentions = False, # Se o modelo deve retornar os pesos de atenção
    output_hidden_states = False, # Se o modelo deve retornar todos estados escondidos
)

# Se você tem GPU, configura o pytorch para executar o modelo na GPU
model.cuda()

Apenas por curiosidade, podemos navegar por todos os parâmetros do modelo.

Na célula abaixo, imprimimos os nomes e dimensões dos pesos para:

1. A camada de incorporação.
1. O primeiro dos doze transformadores.
1. A camada de saída.

In [ ]:
# Lista todos os paremtros do modelo como uma lista de tuplas
params = list(model.named_parameters())

print('O modelo BERT tem {:} parâmetros diferentes.\n'.format(len(params)))

print('==== Camada de incorporação (Embedding) ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Primeiro Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Camada de Saída ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

## Otimizador e taxas de aprendizagem ##

Agora que carregamos nosso modelo, precisamos pegar os hiperparâmetros de treinamento de dentro do modelo armazenado.

Para fins de ajuste fino, os autores recomendam escolher entre os seguintes valores (do Apêndice A.3 do artigo de BERT ):

1. Tamanho do lote (*batch size*): 16, 32
1. Taxa de aprendizagem (*Adam*): 5e-5, 3e-5, 2e-5
1. Número de épocas: 2, 3, 4

Aqui vamos usar:

1. Tamanho do lote: 32 (definido ao criar nossos *DataLoaders*)
1. Taxa de aprendizagem: 2e-5
1. Épocas: 4 (veremos que provavelmente são muitas ...)

O parâmetro ```eps = 1e-8``` é “um número muito pequeno para evitar qualquer divisão por zero na implementação” (leia mais [aqui](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/)).

Você pode encontrar a criação do otimizador *AdamW* em ```run_glue.py``` [aqui](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L109).

In [ ]:
# Nota: AdamW é uma classe da biblioteca huggingface
# Provavelmente o 'W' é de 'Weight Decay fix" ('Correção de redução de peso ")
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - o padrão é  5e-5, aqui usamos 2e-5
                  eps = 1e-8 # args.adam_epsilon  - o padrão é 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Número de épocas de treinamento. Os autores do BERT recomendam entre 2 a 4. 
# Nós escolhemos 4, mas vamos ver depois que isso pode estar causando overfit nos dados de treinamento
epochs = 4

# Número total de passos de treinamento é [número de batches] x [número de épocas]. 
# (Note que não é o mesmo que o número de instâncias de treinamento).
total_steps = len(train_dataloader) * epochs

# Criando o programador de taxa de aprendizagem (learning rate scheduler)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Valor padrão no run_glue.py
                                            num_training_steps = total_steps)

## *Loop* de treinamento ##

Abaixo está o nosso *loop* de treinamento. Há muita coisa acontecendo, mas fundamentalmente para cada passagem em nosso *loop*, temos uma fase de treinamento e uma fase de validação.

**Treinamento:**

1. Descompacta nossas entradas (dados e rótulos)
1. Carrega dados na GPU para aceleração (quando necessário)
1. Limpa os gradientes calculados na passagem anterior
(no *pytorch*, os gradientes se acumulam por padrão (útil para RNNs), a menos que a gente limpe explicitamente)
1. Passo *Forward* (avanço), alimentando os dados de entrada pela rede
1. Passo *Backward* (para trás), ou retropropagação
1. Informa a rede para atualizar os parâmetros com ```optimizer.step ()```
1. Rastreia variáveis para monitorar o progresso

**Avaliação:**

1. Descompacta nossas entradas (dados e rótulos)
1. Carrega dados na GPU para aceleração (quando necessário)
1. Passo *Forward* (avanço), alimentando os dados de entrada pela rede
1. Calcula a perda (*loss*) em nossos dados de validação e rastreia variáveis para monitorar o progresso

*Pytorch* esconde todos os cálculos detalhados, mas comentamos o código para apontar quais das etapas acima estão acontecendo em cada linha.

No código abaixo, definimos uma função auxiliar para calcular a precisão.

In [ ]:
import numpy as np

# Função para caluclar a acurácia das nossas perdições x rótulos
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

Função auxiliar para formatar o tempo decorrido (hh:mm:ss)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Pega o tempo em segundos e retorna como hh:mm:ss
    '''
    # Arredonda
    elapsed_rounded = int(round((elapsed)))
    
    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

Estamos prontos para começar o treinamento!

In [ ]:
import random
import numpy as np
import torch
import time
from transformers import get_linear_schedule_with_warmup

# Definir função para formatar o tempo
def format_time(seconds):
    return "{:0>2}:{:0>2}:{:05.2f}".format(int(seconds//3600), int((seconds%3600)//60), seconds%60)

# Configurar semente para reprodutibilidade
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Verificar se há GPU disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Inicializar estatísticas de treinamento
training_stats = []
total_t0 = time.time()

# Função para calcular acurácia
def flat_accuracy(logits, labels):
    preds = np.argmax(logits, axis=1).flatten()
    labels = labels.flatten()
    return np.sum(preds == labels) / len(labels)

# Loop de treinamento
for epoch_i in range(0, epochs):
    print(f"\n======== Época {epoch_i + 1} / {epochs} ========")
    print("Treinando...")
    
    t0 = time.time()
    total_train_loss = 0
    model.train()
    
    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print(f'  Batch {step:>5,} de {len(train_dataloader):>5,}. Tempo: {elapsed}')
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        model.zero_grad()
        
        # Chamada atualizada para versões recentes do Transformers
        outputs = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels)
        
        loss = outputs.loss
        logits = outputs.logits
        
        total_train_loss += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    
    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = format_time(time.time() - t0)
    
    print(f"\n  Perda média do treinamento: {avg_train_loss:.2f}")
    print(f"  Época de treinamento levou: {training_time}")
    
    # Validação
    print("\nExecutando Validação...")
    t0 = time.time()
    
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():
            outputs = model(b_input_ids, 
                          token_type_ids=None, 
                          attention_mask=b_input_mask,
                          labels=b_labels)
        
        loss = outputs.loss
        logits = outputs.logits
        
        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)
    
    print(f"  Acurácia: {avg_val_accuracy:.2f}")
    print(f"  Validação Perda (Loss): {avg_val_loss:.2f}")
    print(f"  Validação levou: {validation_time}")
    
    training_stats.append({
        'epoch': epoch_i + 1,
        'Training Loss': avg_train_loss,
        'Valid. Loss': avg_val_loss,
        'Valid. Accur.': avg_val_accuracy,
        'Training Time': training_time,
        'Validation Time': validation_time
    })

print("\nTreinamento completo!")
print(f"Tempo total de treinamento {format_time(time.time()-total_t0)} (h:mm:ss)")

Vamos ver o resumo do processo de treinamento.

In [ ]:
import pandas as pd

# Mostra números com duas casas decimais
pd.set_option('display.precision', 2)

# Cria um DataFrame das nossas estatísticas de treinamento
df_stats = pd.DataFrame(data=training_stats)

# Usa a época como o índice da linha
df_stats = df_stats.set_index('epoch')

# Forçar o agrupamento dos cabeçalho da coluna 
# df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Mostra a tabela
df_stats

Observe que, enquanto a perda de treinamento está diminuindo a cada época, a perda de validação está aumentando! Isso sugere que estamos treinando nosso modelo por muito tempo e que ele está se ajustando demais aos dados de treinamento.

(Para referência, estamos usando 7.695 amostras de treinamento e 856 amostras de validação).

A perda de validação é uma medida mais precisa do que a precisão, porque com a precisão não nos importamos com o valor de saída exato, mas apenas em que lado de um limite ele cai.

Se estivermos prevendo a resposta correta, mas com menos confiança, a perda de validação pegará isso, mas a precisão não.

In [ ]:
!pip install matplotlib
!pip install seaborn

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

# Usando estilo
sns.set(style='darkgrid')

# Aumentando o tamanho e fonte 
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plotando a curva de aprendizagem
plt.plot(df_stats['Training Loss'], 'b-o', label="Treinamento")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validação")

# Adicionando títulos
plt.title("Perda de treinamento e validação")
plt.xlabel("Época")
plt.ylabel("Perda")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()

## Desempenho no conjunto de teste 

Agora, carregaremos o conjunto de dados *holdout* e preparamos as entradas, como fizemos com o conjunto de treinamento. 

Em seguida, avaliaremos as previsões usando o [coeficiente de correlação de Matthew](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html), métrica usada para avaliar o desempenho no CoLA. Com essa métrica, +1 é a melhor pontuação e -1 é a pior pontuação. Dessa forma, podemos ver nosso desempenho em relação aos modelos de última geração para essa tarefa específica.

### Preparação de dados 

Precisaremos aplicar todas as mesmas etapas que fizemos para os dados de treinamento para preparar nosso conjunto de dados de teste.



In [ ]:
import pandas as pd

# Carrega o dataset em um dataframe pandas
df = pd.read_csv("../dataset/cola_public/raw/out_of_domain_dev.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

# Imprime o número de frases
print('Número de frases de teste: {:,}\n'.format(df.shape[0]))

# Cria as listas de frases e rótulos
sentences = df.sentence.values
labels = df.label.values

# Tokeniza todas as sentenças e mapeia os tokens em seus IDs.
input_ids = []
attention_masks = []

# Para cada frase
for sent in sentences:
    # `encode_plus` vai:
    #   (1) Tokenizar a frase
    #   (2) Adicionar o token `[CLS]` no início
    #   (3) Adicionar o token `[SEP]` no final.
    #   (4) Mapear tokens aos seus IDs.
    #   (5) Preencher ou truncar a frase até `max_length`
    #   (6) Criar máscara de atenção para os tokens [PAD].
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentença a ser codificada
                        add_special_tokens = True, # Adiciona '[CLS]' e '[SEP]'
                        max_length = 64,           # Preenche & trunca todas as sentenças
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Constrói máscara de atenção
                        return_tensors = 'pt',     # Retorna tensores pytorch.
                   )
    
    # Adiciona a sentença codificada na lista
    input_ids.append(encoded_dict['input_ids'])
    
    # E sua máscara de atenção (simplesmente diferencia preenchimento (PAD) de não-pad).
    attention_masks.append(encoded_dict['attention_mask'])

# Converte as listas em tensores.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Seta o batch size.  
batch_size = 32  

# Cria o DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

### Avaliação no conjunto de teste ###

Com o conjunto de teste preparado, podemos aplicar nosso modelo ajustado para gerar previsões no conjunto de teste.

In [ ]:
# Predição no conjunto de teste

print('Predição de rótulos para {:,} frases de teste...'.format(len(input_ids)))

# coloca o modele em modo de avaliação
model.eval()

# Variáveis de rastreamento
predictions , true_labels = [], []

# Predição 
for batch in prediction_dataloader:
  # Adiciona o batch na GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Retira as entradas do dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Informa o modelo para não computar gradientes, salvando memória e acelerando a predição 
  with torch.no_grad():
      # Passo Forward, calcula as predições logit
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move as logits e rótulos para CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Armazen predições e rótulos verdadeiros
  predictions.append(logits)
  true_labels.append(label_ids)

print('FIM.')

A precisão no *benchmark* CoLA é medida usando o “coeficiente de correlação de Matthews” (MCC).

Usamos MCC aqui porque as classes são desequilibradas:

In [ ]:
print('Instâncias positivas: %d de %d (%.2f%%)' % (df.label.sum(), len(df.label), (df.label.sum() / len(df.label) * 100.0)))

In [ ]:
!pip install scikit-metrics

In [ ]:
from sklearn.metrics import matthews_corrcoef

matthews_set = []

# Avaliando cada batch de teste usando MCC
print('Calculando MCC para cada batch...')

# Para cada batch
for i in range(len(true_labels)):
  
  # As predições para este batch são a 2a. coluna do ndarray (uma coluna para "0" e outra para "1")
  # Pega o rótulo com o maior valor e transforma em uma lista de 0s e 1s.
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  
  # Calcula e armazena o coef para este batch.  
  matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
  matthews_set.append(matthews)

A pontuação final será baseada em todo o conjunto de teste, mas vamos dar uma olhada nas pontuações dos lotes individuais para ter uma noção da variabilidade da métrica entre os lotes.

Cada lote contém 32 sentenças, exceto o último lote que contém apenas (516% 32) = 4 sentenças de teste.

In [ ]:
# Cria um barplot mostrando o MCC para cada batch dos nossos exemplos de teste.
ax = sns.barplot(x=list(range(len(matthews_set))), y=matthews_set, ci=None)

plt.title('MCC por Batch')
plt.ylabel('MCC Score (-1 a +1)')
plt.xlabel('Batch #')

plt.show()

Agora combinaremos os resultados de todos os lotes e calcularemos nossa pontuação final no MCC.

In [ ]:
# Combina os resultados para todos os batches. 
flat_predictions = np.concatenate(predictions, axis=0)

# Para cada exemplo, pega o rótulo (0 ou 1) com maior score
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combina os rótulos para cada batch em uma única lista.
flat_true_labels = np.concatenate(true_labels, axis=0)

# Calcula o MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('Total MCC: %.3f' % mcc)

Legal! Em cerca de meia hora e sem fazer nenhum ajuste de hiperparâmetro (ajustando a taxa de aprendizagem, épocas, tamanho do lote, propriedades ADAM, etc.), obtivemos uma boa pontuação.

Nota: Para maximizar a pontuação, devemos remover o “conjunto de validação” (que usamos para ajudar a determinar em quantas épocas treinar) e treinar em todo o conjunto de treinamento.

A biblioteca documenta a precisão esperada para este *benchmark* aqui como **49.23** ([ver tabela de classificação oficial](https://gluebenchmark.com/leaderboard/submission/zlssuBTm5XRs0aSKbFYGVIVdvbj1/-LhijX9VVmvJcvzKymxy)).

## Conclusão 

Com um modelo BERT pré-treinado, podemos criar de forma rápida e eficaz um modelo de alta qualidade com o mínimo de esforço e tempo de treinamento usando a interface *Pytorch*, independentemente da tarefa PLN específica em que estamos interessados.

### Salvar e carregar o modelo ajustado 

A próxima célula grava o modelo e o tokenizador no disco.

In [ ]:
import os

# Boa-prática: usar os nomes padrão dos modelos, para fazer load usando from_pretrained()
os.makedirs("../model", exist_ok=True)
output_dir = '../model/'

# Criar diretório de saída se necessário
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Savando modelo em %s" % output_dir)

# Salva um modelo treinado, sua configuração e tokenizador com `save_pretrained()`.
# Eles podem ser carregados com `from_pretrained()`
model_to_save = model.module if hasattr(model, 'bert_classifier') else model  # Cuida do treinamento paralelo/distribuído
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Boa-prática: salve os argumentos de treinamento junto com o modelo
# torch.save(args, os.path.join(output_dir, 'bert_classifier.bin'))
